# What are free variables and closures?

- **Recall**: a function defined inside another can access the non-local variables

In [1]:
def outer():
    x = 'python'
    def inner():
        print(x)
    inner()

In [2]:
outer()

python


- When the `outer` function is compiled, Python recognizes that `x` is a local variable
    - Then, when we call `outer`, `x` is created in the local scope of `outer`
- But also, when we define `inner` inside of `outer`, we don't define a separate `x` inside the function
    - Therefore, Python recognizes that it needs to look elsewhere, and finds `x` in the non-local scope of `outer`
        - This `x` is an example of a **free variable**

- When we decompose the `inner` function, we have:
    1. The actual function
    2. The free variable `x`
    
- The way these two things are bound together is called a **closure**
    - `inner` **encloses** the free variable `x`

# What would happen if we returned `inner` instead of calling it?

In [3]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [4]:
outer()

<function __main__.outer.<locals>.inner()>

- As expected, we get a function object back
    - `inner` still encloses `x`
        - **Therefore, we're not just getting a function- we're getting the closure!**

In [5]:
fn = outer()
fn()

python


- But wait a minute
    - When `outer` was compiled, `inner` was defined
        - So, `x` was recognized as a non-local variable
            - However, `x` was never evaluated
                - Therefore, *how did Python know it was equal to `'python'`?*
                    - The `outer` function was already done running
                    - Its scope should have been deleted

- The value of `x` is shared between two scopes
    1. the `outer` scope
    2. the `closure` scope
- We call `x` a **multi-scoped variable**


- *How does Python do this?*
    - By creating a **cell**
        - When `x` is recognized to be a non-local (i.e. free) variable, Python creates a cell
            - Both `outer.x` and `inner.x` point to the same cell now
                - The cell then corresponds to the memory location where `'python'` is stored
                    - That way, when `outer` is finished running and `outer.x` is deleted, the cell still exists

# How can we use introspection to see the free variables and closures?

- We can use `__code__.co_freevars` and `__closure__`

In [6]:
def outer():
    a = 100
    x = 'python'
    def inner():
        a = 10
        print(f'{x} rocks!')
    return inner

In [7]:
fn = outer()

In [8]:
fn.__code__.co_freevars

('x',)

- As we can see, we got `x`
    - *But what about `a`?*
        - Not a free variable!
            - Both versions of `a` are local

In [9]:
fn.__closure__

(<cell at 0x000002D704FE0438: str object at 0x000002D702FD8830>,)

- This returned:
    1. The memory address of the cell
    2. The memory address of the object

# Can we modify the free variables?

In [10]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [11]:
fn = counter()
fn()

1

- *What if we call it again?*

In [12]:
fn()

2

- As we can see, it doesn't re-define `count=0` when we call `fn`

# What if we create multiple instances of `fn`?

In [13]:
f1 = counter()
f2 = counter()

In [14]:
f1()

1

In [15]:
f1()

2

In [16]:
f2()

1

In [17]:
f2()

2

- As we can see, **two separate cells were created**
    - One for each function

- Each time we create a function, a new closure is created

In [18]:
f1.__closure__, f2.__closure__

((<cell at 0x000002D704FE0408: int object at 0x00007FFCB0FD7C60>,),
 (<cell at 0x000002D704FE07C8: int object at 0x00007FFCB0FD7C60>,))

- As we can see, they're two distinct cells
    - *But why do they point to the same memory address?*
        - Because they both have value 2

In [19]:
f2()

3

In [20]:
f1.__closure__, f2.__closure__

((<cell at 0x000002D704FE0408: int object at 0x00007FFCB0FD7C60>,),
 (<cell at 0x000002D704FE07C8: int object at 0x00007FFCB0FD7C80>,))

- Now they're point to different memory addresses

# Can extended scopes be shared?

In [21]:
def outer():
    
    count = 0
    
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

In [22]:
f1, f2 = outer()

In [23]:
f1.__closure__, f2.__closure__

((<cell at 0x000002D70502F318: int object at 0x00007FFCB0FD7C20>,),
 (<cell at 0x000002D70502F318: int object at 0x00007FFCB0FD7C20>,))

- As we can see, the closures for `f1` and `f2` contain the same cell
    - This is because they both point to `count` in the scope of `outer`

In [24]:
f1()

1

In [25]:
f2()

2

- *How often do we share extended scopes?*
    - Pretty often
        - But often by mistake

In [26]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [27]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)

In [28]:
add_1(1)

2

In [29]:
add_2(1)

3

In [30]:
add_3(1)

4

- These are three separate closures
    - No shared scope

In [31]:
add_1.__closure__, add_2.__closure__, add_3.__closure__

((<cell at 0x000002D704FE0678: int object at 0x00007FFCB0FD7C40>,),
 (<cell at 0x000002D704FE0F78: int object at 0x00007FFCB0FD7C60>,),
 (<cell at 0x000002D704FE0378: int object at 0x00007FFCB0FD7C80>,))

- But let's say we tried to make it more convenient by writing a loop:

In [32]:
list_of_adders = []

for n in range(1, 4):
    list_of_adders.append(lambda x: x + n)

- But here, `n` isn't being redefined in each loop
    - It's simply being updated

In [33]:
add_1, add_2, add_3 = list_of_adders

In [34]:
add_1(1)

4

In [35]:
add_1.__closure__, add_2.__closure__, add_3.__closure__

(None, None, None)

- As we can see, this call returned the wrong answer
    - This is because the `n` object was the same for each function

___

- **Aside**: people often think that lambdas are closures
    - They're not!
        - They simply create functions
            - To become a closure, they require a free variable
            
___